# Generating Session Summary with LLMs

This project utilizes LlamaIndex and AI technology to analyze Formula 1 car data and generate session summaries. It is particularly beneficial for race engineers seeking detailed insights and performance analysis during races.


In [4]:
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

True

## Optional: Setup Observability

Here we will use our Arize Phoenix integration to view traces through the query engine. It will be available at http://localhost:6006


In [8]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
import phoenix as px

px.launch_app()


endpoint = "http://127.0.0.1:6006/v1/traces"  # Phoenix receiver address

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


### Define the LLM and the Embedding


In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

### Start SQL Database


In [6]:
from sqlalchemy import create_engine
from llama_index.core import SQLDatabase

engine = create_engine('sqlite:///Spain_2024_FP1.db')

sql_database = SQLDatabase(engine)

# Part 1: Text-to-SQL Query Engine

Once we have constructed our SQL database, we can use the `NLSQLTableQueryEngine` to construct natural language queries that are synthesized into SQL queries.

Note that we need to specify the tables we want to use with this query engine. If we don't the query engine will pull all the schema context, which could overflow the context window of the LLM.


In [9]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

hamilton_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["hamilton"])
query_str = "This database contains the data for the driver HAM in the first free practice, based on this data provide a short overview on how was his speed and time taking into account his compound"
response = hamilton_query_engine.query(query_str)

display(Markdown(f"{response}"))

In the first free practice session, driver HAM started on the HARD compound. His speed varied throughout the session, with his fastest lap being 77.632 seconds on lap 2. He showed consistent speed on the straights, reaching up to 316 km/h, but struggled with lower speeds in some corners. Overall, his performance on the HARD compound seemed solid, with room for improvement in certain areas.

This query engine should be used in any case where you can specify the tables you want to query over beforehand, or the total size of all the table schema plus the rest of the prompt fits your context window.

# Part 2: Query-Time Retrieval of Tables for Text-to-SQL

If we don't know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the `SQLTableNodeMapping` object, which takes in a `SQLDatabase` and produces a Node object for each `SQLTableSchema` object passed into the `ObjectIndex` constructor.


In [ ]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="hamilton", context_str="This table contains the lap time for the first Hamilton's free practice at Spain."))]

obj_index = ObjectIndex.from_objects(objects=table_schema_objs,
                                     object_mapping=table_node_mapping,
                                     index_cls=VectorStoreIndex)

table_retriever = obj_index.as_retriever(similarity_top_k=1)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database=sql_database, table_retriever=table_retriever)

### Define the Retriever and the Query Engine


In [7]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["mercedes"], return_raw=True
)

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)